In [1]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [2]:
import os
if not os.path.isdir('saved_models_mnist_{}'.format('bigan')):
    os.mkdir('saved_models_mnist_{}'.format('bigan'))
f = open('saved_models_mnist_{}/log_collapse1.txt'.format('bigan'), mode='w')
import torch.utils.data as Data

from keras.datasets import mnist,fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K
import os
import matplotlib.pyplot as plt

import numpy as np

class BIGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # Build the encoder
        self.encoder = self.build_encoder()

        # The part of the bigan that trains the discriminator and encoder
        self.discriminator.trainable = False

        # Generate image from sampled noise
        z = Input(shape=(self.latent_dim, ))
        img_ = self.generator(z)

        # Encode image
        img = Input(shape=self.img_shape)
        z_ = self.encoder(img)

        # Latent -> img is fake, and img -> latent is valid
        fake = self.discriminator([z, img_])
        valid = self.discriminator([z_, img])

        # Set up and compile the combined model
        # Trains generator to fool the discriminator
        self.bigan_generator = Model([z, img], [fake, valid])
        self.bigan_generator.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
            optimizer=optimizer)


    def build_encoder(self):
        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.latent_dim))

        model.summary()

        img = Input(shape=self.img_shape)
        z = model(img)

        return Model(img, z)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(512, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        z = Input(shape=(self.latent_dim,))
        gen_img = model(z)

        return Model(z, gen_img)

    def build_discriminator(self):

        z = Input(shape=(self.latent_dim, ))
        img = Input(shape=self.img_shape)
        d_in = concatenate([z, Flatten()(img)])

        model = Dense(1024)(d_in)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        model = Dense(1024)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        model = Dense(1024)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        validity = Dense(1, activation="sigmoid")(model)

        return Model([z, img], validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (X_test,y_test) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_test = X_test / 127.5 - 1.
        X_test = np.expand_dims(X_test, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]
                z = np.random.normal(size=(batch_size, self.latent_dim))
                imgs_ = self.generator.predict(z)

                # Select a random batch of images and encode
                # idx = np.random.randint(0, X_train.shape[0], batch_size)
                # imgs = X_train[idx]
                z_ = self.encoder.predict(image_batch)

                # Train the discriminator (img -> z is valid, z -> img is fake)
                d_loss_real = self.discriminator.train_on_batch([z_, image_batch], valid)
                d_loss_fake = self.discriminator.train_on_batch([z, imgs_], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator (z -> img is valid and img -> z is is invalid)
                g_loss = self.bigan_generator.train_on_batch([z, image_batch], [valid, fake])

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch,global_step, d_loss[0],
                                                                                   100 * d_loss[1], g_loss[0]))

                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sample_num):
        r, c = sample_num//10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
#         sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')
if __name__ == '__main__':
    bigan = BIGAN()
    bigan.train(epochs=30, batch_size=64, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               51712     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_7 (Dense)              (None, 784)               402192    
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 0.867266, acc: 41.41%] [G loss: 4.966393]
epoch:0 step:2 [D loss: 0.424162, acc: 74.22%] [G loss: 5.270675]
epoch:0 step:3 [D loss: 0.170252, acc: 98.44%] [G loss: 5.608031]
epoch:0 step:4 [D loss: 0.131281, acc: 98.44%] [G loss: 7.154961]
epoch:0 step:5 [D loss: 0.072370, acc: 99.22%] [G loss: 9.180442]
epoch:0 step:6 [D loss: 0.042065, acc: 100.00%] [G loss: 9.376972]
epoch:0 step:7 [D loss: 0.035580, acc: 100.00%] [G loss: 10.704073]
epoch:0 step:8 [D loss: 0.019201, acc: 100.00%] [G loss: 10.631642]
epoch:0 step:9 [D loss: 0.018400, acc: 100.00%] [G loss: 11.233311]
epoch:0 step:10 [D loss: 0.014424, acc: 100.00%] [G loss: 11.991962]
epoch:0 step:11 [D loss: 0.014868, acc: 100.00%] [G loss: 12.082867]
epoch:0 step:12 [D loss: 0.008434, acc: 100.00%] [G loss: 12.335401]
epoch:0 step:13 [D loss: 0.011807, acc: 100.00%] [G loss: 13.163174]
epoch:0 step:14 [D loss: 0.012628, acc: 100.00%] [G loss: 12.675383]
epoch:0 step:15 [D loss: 0.007499, acc: 100.00%] [G lo

epoch:0 step:122 [D loss: 0.380156, acc: 79.69%] [G loss: 9.781399]
epoch:0 step:123 [D loss: 0.238576, acc: 87.50%] [G loss: 9.530716]
epoch:0 step:124 [D loss: 0.251878, acc: 89.06%] [G loss: 9.489276]
epoch:0 step:125 [D loss: 0.226107, acc: 87.50%] [G loss: 8.816999]
epoch:0 step:126 [D loss: 0.263031, acc: 89.84%] [G loss: 9.181766]
epoch:0 step:127 [D loss: 0.247813, acc: 88.28%] [G loss: 9.568509]
epoch:0 step:128 [D loss: 0.237147, acc: 86.72%] [G loss: 9.461643]
epoch:0 step:129 [D loss: 0.168317, acc: 94.53%] [G loss: 9.462431]
epoch:0 step:130 [D loss: 0.291471, acc: 84.38%] [G loss: 9.225388]
epoch:0 step:131 [D loss: 0.186655, acc: 92.97%] [G loss: 8.702338]
epoch:0 step:132 [D loss: 0.196270, acc: 89.06%] [G loss: 8.918526]
epoch:0 step:133 [D loss: 0.317965, acc: 87.50%] [G loss: 10.272030]
epoch:0 step:134 [D loss: 0.142425, acc: 92.97%] [G loss: 9.430574]
epoch:0 step:135 [D loss: 0.272644, acc: 86.72%] [G loss: 9.087436]
epoch:0 step:136 [D loss: 0.287575, acc: 86.72%

...

epoch:29 step:27635 [D loss: 0.648529, acc: 64.84%] [G loss: 1.874983]
epoch:29 step:27636 [D loss: 0.625994, acc: 62.50%] [G loss: 1.755139]
epoch:29 step:27637 [D loss: 0.629152, acc: 66.41%] [G loss: 1.836180]
epoch:29 step:27638 [D loss: 0.639355, acc: 64.84%] [G loss: 1.957454]
epoch:29 step:27639 [D loss: 0.699125, acc: 53.12%] [G loss: 1.915511]
epoch:29 step:27640 [D loss: 0.612779, acc: 70.31%] [G loss: 1.880834]
epoch:29 step:27641 [D loss: 0.633059, acc: 67.19%] [G loss: 2.142483]
epoch:29 step:27642 [D loss: 0.623544, acc: 67.97%] [G loss: 2.093732]
epoch:29 step:27643 [D loss: 0.615153, acc: 69.53%] [G loss: 2.120329]
epoch:29 step:27644 [D loss: 0.567804, acc: 75.00%] [G loss: 2.229023]
epoch:29 step:27645 [D loss: 0.618628, acc: 62.50%] [G loss: 2.116916]
epoch:29 step:27646 [D loss: 0.650621, acc: 64.06%] [G loss: 1.786906]
epoch:29 step:27647 [D loss: 0.677111, acc: 54.69%] [G loss: 1.837225]
epoch:29 step:27648 [D loss: 0.701613, acc: 55.47%] [G loss: 1.886350]
epoch:

epoch:29 step:27755 [D loss: 0.618767, acc: 70.31%] [G loss: 1.819257]
epoch:29 step:27756 [D loss: 0.671513, acc: 59.38%] [G loss: 1.744869]
epoch:29 step:27757 [D loss: 0.662041, acc: 54.69%] [G loss: 1.712209]
epoch:29 step:27758 [D loss: 0.684771, acc: 62.50%] [G loss: 1.864754]
epoch:29 step:27759 [D loss: 0.706421, acc: 56.25%] [G loss: 1.632470]
epoch:29 step:27760 [D loss: 0.658163, acc: 60.94%] [G loss: 1.774436]
epoch:29 step:27761 [D loss: 0.699265, acc: 58.59%] [G loss: 1.887412]
epoch:29 step:27762 [D loss: 0.643842, acc: 63.28%] [G loss: 1.791494]
epoch:29 step:27763 [D loss: 0.615624, acc: 67.97%] [G loss: 1.902063]
epoch:29 step:27764 [D loss: 0.601863, acc: 64.84%] [G loss: 1.755997]
epoch:29 step:27765 [D loss: 0.673723, acc: 64.06%] [G loss: 1.816717]
epoch:29 step:27766 [D loss: 0.607386, acc: 65.62%] [G loss: 1.775560]
epoch:29 step:27767 [D loss: 0.678368, acc: 58.59%] [G loss: 1.725131]
epoch:29 step:27768 [D loss: 0.649103, acc: 62.50%] [G loss: 1.853278]
epoch:

epoch:29 step:27870 [D loss: 0.613452, acc: 60.94%] [G loss: 1.882453]
epoch:29 step:27871 [D loss: 0.672931, acc: 60.16%] [G loss: 1.793445]
epoch:29 step:27872 [D loss: 0.634717, acc: 63.28%] [G loss: 1.868276]
epoch:29 step:27873 [D loss: 0.670552, acc: 56.25%] [G loss: 1.829442]
epoch:29 step:27874 [D loss: 0.631478, acc: 66.41%] [G loss: 1.937310]
epoch:29 step:27875 [D loss: 0.602139, acc: 65.62%] [G loss: 1.873680]
epoch:29 step:27876 [D loss: 0.710085, acc: 57.03%] [G loss: 1.809974]
epoch:29 step:27877 [D loss: 0.653200, acc: 59.38%] [G loss: 1.775785]
epoch:29 step:27878 [D loss: 0.644987, acc: 58.59%] [G loss: 1.810421]
epoch:29 step:27879 [D loss: 0.649844, acc: 62.50%] [G loss: 1.980637]
epoch:29 step:27880 [D loss: 0.603068, acc: 68.75%] [G loss: 2.034883]
epoch:29 step:27881 [D loss: 0.626201, acc: 63.28%] [G loss: 1.834467]
epoch:29 step:27882 [D loss: 0.621639, acc: 68.75%] [G loss: 1.911426]
epoch:29 step:27883 [D loss: 0.640539, acc: 64.84%] [G loss: 1.866541]
epoch:

epoch:29 step:27990 [D loss: 0.684691, acc: 56.25%] [G loss: 1.817436]
epoch:29 step:27991 [D loss: 0.676021, acc: 57.03%] [G loss: 1.799149]
epoch:29 step:27992 [D loss: 0.634606, acc: 64.06%] [G loss: 1.855769]
epoch:29 step:27993 [D loss: 0.691691, acc: 56.25%] [G loss: 1.675019]
epoch:29 step:27994 [D loss: 0.681356, acc: 56.25%] [G loss: 1.709130]
epoch:29 step:27995 [D loss: 0.650272, acc: 60.94%] [G loss: 1.853742]
epoch:29 step:27996 [D loss: 0.615344, acc: 67.97%] [G loss: 1.881715]
epoch:29 step:27997 [D loss: 0.686544, acc: 59.38%] [G loss: 1.809553]
epoch:29 step:27998 [D loss: 0.643269, acc: 60.94%] [G loss: 1.998341]
epoch:29 step:27999 [D loss: 0.687668, acc: 54.69%] [G loss: 1.793290]
epoch:29 step:28000 [D loss: 0.638199, acc: 62.50%] [G loss: 1.781586]
##############
[2.33132682 1.34286611 6.14987486 4.79874249 3.5009655  5.35673421
 4.33827509 4.6816606  4.49754165 3.71467231]
##########
epoch:29 step:28001 [D loss: 0.733953, acc: 50.78%] [G loss: 1.884341]
epoch:29 

epoch:29 step:28105 [D loss: 0.633867, acc: 62.50%] [G loss: 1.950405]
epoch:29 step:28106 [D loss: 0.648518, acc: 62.50%] [G loss: 1.786213]
epoch:29 step:28107 [D loss: 0.651119, acc: 58.59%] [G loss: 1.955636]
epoch:29 step:28108 [D loss: 0.590074, acc: 65.62%] [G loss: 1.808781]
epoch:29 step:28109 [D loss: 0.581717, acc: 70.31%] [G loss: 1.975477]
epoch:29 step:28110 [D loss: 0.637209, acc: 62.50%] [G loss: 2.395953]
